# 4. SALT2 fits (sncosmo)

Fits a **SALT2** model to each light curve using **sncosmo**. For each object:

- **Inputs:** Light-curve CSV (MJD, filter g/r, forced_ujy, forced_ujy_error; zp=23.9, zpsys=ab), plus from `ztf_cleansed.csv`: redshift (or host_redshift), and A_V → mwebv = A_V/3.1 (Milky Way dust).
- **Fitted parameters:** t0, x0, x1, c (z and mwebv fixed).
  - **t0:** time of B-band maximum (MJD).
  - **x0:** flux normalization (overall scale).
  - **x1:** stretch (positive = broader light curve).
  - **c:** colour (positive = redder).

**Output:** `runs/<run>/sncosmo_fits.csv` - columns include ztf_id, redshift, ncall, ndof, chisq, t0, x0, x1, c.

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import sncosmo
from astropy.table import Table
import matplotlib.pyplot as plt
import warnings

project_root = Path.cwd().parent
print(f"Project root: {project_root}")

Project root: /Users/david/Code/msc


In [2]:
# User input: run folder name
folder_name = input("Enter the run folder name: ").strip()
run_folder = project_root / "runs" / folder_name

# get all light curve for the run
lightcurve_files = sorted(run_folder.glob("*_lightcurve.csv"))
print(f"Found {len(lightcurve_files)} lightcurve CSV in {run_folder}")
for f in lightcurve_files:
    print(f"{f.name}")

Enter the run folder name: run1
Found 5003 lightcurve CSV in /Users/david/Code/msc/runs/run1
ZTF17aabtvsy_lightcurve.csv
ZTF17aabvong_lightcurve.csv
ZTF17aacldgo_lightcurve.csv
ZTF17aadlxmv_lightcurve.csv
ZTF18aaaonon_lightcurve.csv
ZTF18aaaooqj_lightcurve.csv
ZTF18aaaqexr_lightcurve.csv
ZTF18aadlaxo_lightcurve.csv
ZTF18aadxnul_lightcurve.csv
ZTF18aadzfso_lightcurve.csv
ZTF18aaeqjmc_lightcurve.csv
ZTF18aaermez_lightcurve.csv
ZTF18aafdigb_lightcurve.csv
ZTF18aagkwgz_lightcurve.csv
ZTF18aagtwyh_lightcurve.csv
ZTF18aahfbqp_lightcurve.csv
ZTF18aahtjsc_lightcurve.csv
ZTF18aahvndq_lightcurve.csv
ZTF18aailmnv_lightcurve.csv
ZTF18aaisqmw_lightcurve.csv
ZTF18aaiwzie_lightcurve.csv
ZTF18aaiykoz_lightcurve.csv
ZTF18aaizerg_lightcurve.csv
ZTF18aajivpr_lightcurve.csv
ZTF18aajkcdn_lightcurve.csv
ZTF18aajkrxi_lightcurve.csv
ZTF18aajpjdi_lightcurve.csv
ZTF18aajvqye_lightcurve.csv
ZTF18aakitiq_lightcurve.csv
ZTF18aakiwbs_lightcurve.csv
ZTF18aaklpdo_lightcurve.csv
ZTF18aamvfeb_lightcurve.csv
ZTF18aamxad

In [ ]:
ztf_cleansed_path = project_root / "ztf_cleansed.csv"
ztf_df = pd.read_csv(ztf_cleansed_path)

# Initialize list to collect results
results_list = []

for idx, lc_path in enumerate(lightcurve_files, 1):
    print(f"Processing file {idx}/{len(lightcurve_files)}: {lc_path.name}")
    lc_df = pd.read_csv(lc_path)

    # clean lightcurve data ready for sncosmo
    
    lc_df['MJD'] = pd.to_numeric(lc_df['MJD'], errors='coerce')
    # we will only use 'forced_ujy', 'forced_ujy_error'
    lc_df = lc_df.dropna(subset=['MJD', 'filter', 'forced_ujy', 'forced_ujy_error'])
    lc_df['filter'] = lc_df['filter'].astype(str).str.strip().str.lower()
    lc_df = lc_df[lc_df['forced_ujy'] > 0]
    lc_df = lc_df[lc_df['forced_ujy_error'] > 0]
    lc_df = lc_df.sort_values('MJD')

    bands = lc_df['filter'].map({'g': 'ztfg', 'r': 'ztfr'}).values

    data = Table({
        'time': lc_df['MJD'].values,
        'band': bands,
        'flux': lc_df['forced_ujy'].values,
        'fluxerr': lc_df['forced_ujy_error'].values,
        'zp': np.full(len(lc_df), 23.9),
        'zpsys': np.array(['ab'] * len(lc_df)),
    })
    # print(set(bands))
    # print(data)
    print(f"Loaded {len(data)} data points after cleaning.")

    # add dust (commented out for now)
    # dust = sncosmo.CCM89Dust()
    # Rv = 3.1 # Milky Way extinction law

    model = sncosmo.Model(source="salt2")
    obj_id = lc_df['ztf_id'].iloc[0]

    z_row = ztf_df.loc[ztf_df['ZTFID'] == obj_id]
    if z_row.empty:
        print(f"  No row found in ztf_cleansed for ZTFID {obj_id}, skipping.")
        continue
    # MW dust (commented out for now)
    # A_V = float(z_row['A_V'].iloc[0])
    # mwebv = A_V / Rv
    
    # use host redshift from the TNS     
    z_ztf = z_row['host_redshift'].iloc[0]

    # Initial t0 from ztf_cleansed peak time (peakt) which we already converted to MJD, fallback to first MJD in light curve
    # peakt = pd.to_numeric(z_row['peakt'].iloc[0], errors='coerce')
    # t0_guess = peakt if np.isfinite(peakt) else lc_df['MJD'].min()

    # Model.set with t0 initial guess
    # model.set(z=z_ztf, t0=t0_guess)
    model.set(z=z_ztf)

    try:
        with warnings.catch_warnings(record=True) as w:
            warnings.simplefilter("always", RuntimeWarning)
            result, fitted_model = sncosmo.fit_lc(
                data, model,
                ['t0', 'x0', 'x1', 'c'])
            has_numerical_warning = any(
                isinstance(x.message, RuntimeWarning) for x in w
            )
        # print(result)
        # Collect results
        params = dict(zip(result.param_names, result.parameters))
        result_dict = {
            'ztf_id': obj_id,
            'redshift': z_ztf,
            'ncall': result.ncall,
            'ndof': result.ndof,
            'chisq': result.chisq,
            't0': params['t0'],
            'x0': params['x0'],
            'x1': params['x1'],
            'c': params['c'],
            'fit_numerical_warning': has_numerical_warning,
        }
        results_list.append(result_dict)

        print(f"  Fit completed for {obj_id}" + (" (numerical warning)" if has_numerical_warning else ""))
    except RuntimeError as e:
        print(f"  Fit failed for {obj_id}: {e}")
        continue

    # Create and save plot for each lightcurve fit
    # fig, ax = plt.subplots(figsize=(8, 5))
    # sncosmo.plot_lc(data, model=fitted_model, ax=ax, errors=result.errors if hasattr(result, 'errors') else None)
    # ax.set_title(f"Light Curve Fit: {obj_id}")
    # ax.set_xlabel('MJD')
    # ax.set_ylabel('Flux (uJy)')
    # plot_path = run_folder / f"{obj_id}_plot.png"
    # plt.tight_layout()
    # plt.savefig(plot_path)
    # plt.close(fig)
    # print(f"  Plot saved to {plot_path}")

# After loop, create dataframe and save to CSV
results_df = pd.DataFrame(results_list)
output_csv_path = run_folder / "sncosmo_fits.csv"
results_df.to_csv(output_csv_path, index=False)
print(f"Results saved to {output_csv_path}")

Processing file 1/5003: ZTF17aabtvsy_lightcurve.csv
Loaded 23 data points after cleaning.
  Fit completed for ZTF17aabtvsy
Processing file 2/5003: ZTF17aabvong_lightcurve.csv
Loaded 31 data points after cleaning.
  Fit completed for ZTF17aabvong
Processing file 3/5003: ZTF17aacldgo_lightcurve.csv
Loaded 17 data points after cleaning.
  Fit completed for ZTF17aacldgo
Processing file 4/5003: ZTF17aadlxmv_lightcurve.csv
Loaded 28 data points after cleaning.
  Fit completed for ZTF17aadlxmv
Processing file 5/5003: ZTF18aaaonon_lightcurve.csv
Loaded 13 data points after cleaning.
  Fit completed for ZTF18aaaonon
Processing file 6/5003: ZTF18aaaooqj_lightcurve.csv
Loaded 41 data points after cleaning.
  Fit completed for ZTF18aaaooqj
Processing file 7/5003: ZTF18aaaqexr_lightcurve.csv
Loaded 11 data points after cleaning.
  Fit completed for ZTF18aaaqexr
Processing file 8/5003: ZTF18aadlaxo_lightcurve.csv
Loaded 20 data points after cleaning.
  Fit completed for ZTF18aadlaxo
Processing file 

  Fit completed for ZTF18aatezaj
Processing file 69/5003: ZTF18aatgdph_lightcurve.csv
Loaded 93 data points after cleaning.
  Fit completed for ZTF18aatgdph
Processing file 70/5003: ZTF18aatjfoe_lightcurve.csv
Loaded 38 data points after cleaning.
  Fit completed for ZTF18aatjfoe
Processing file 71/5003: ZTF18aatmlsy_lightcurve.csv
Loaded 39 data points after cleaning.
  Fit completed for ZTF18aatmlsy
Processing file 72/5003: ZTF18aatrkga_lightcurve.csv
Loaded 59 data points after cleaning.
  Fit completed for ZTF18aatrkga
Processing file 73/5003: ZTF18aatrtkc_lightcurve.csv
Loaded 70 data points after cleaning.
  Fit completed for ZTF18aatrtkc
Processing file 74/5003: ZTF18aatwqed_lightcurve.csv
Loaded 30 data points after cleaning.
  Fit completed for ZTF18aatwqed
Processing file 75/5003: ZTF18aauizcr_lightcurve.csv
Loaded 392 data points after cleaning.
  Fit completed for ZTF18aauizcr
Processing file 76/5003: ZTF18aaunfqq_lightcurve.csv
Loaded 18 data points after cleaning.
  Fit c

  Fit completed for ZTF18abckujq
Processing file 138/5003: ZTF18abcrtnq_lightcurve.csv
Loaded 12 data points after cleaning.
  Fit completed for ZTF18abcrtnq
Processing file 139/5003: ZTF18abcrxoj_lightcurve.csv
Loaded 37 data points after cleaning.
  Fit completed for ZTF18abcrxoj
Processing file 140/5003: ZTF18abcsgvj_lightcurve.csv
Loaded 11 data points after cleaning.
  Fit completed for ZTF18abcsgvj
Processing file 141/5003: ZTF18abcvush_lightcurve.csv
Loaded 12 data points after cleaning.
  Fit completed for ZTF18abcvush
Processing file 142/5003: ZTF18abcysdx_lightcurve.csv
Loaded 11 data points after cleaning.
  Fit completed for ZTF18abcysdx
Processing file 143/5003: ZTF18abdbuty_lightcurve.csv
Loaded 16 data points after cleaning.
  Fit completed for ZTF18abdbuty
Processing file 144/5003: ZTF18abdcudb_lightcurve.csv
Loaded 70 data points after cleaning.
  Fit completed for ZTF18abdcudb
Processing file 145/5003: ZTF18abddrhd_lightcurve.csv
Loaded 5 data points after cleaning.
 

  Fit completed for ZTF18abmjvuf
Processing file 206/5003: ZTF18abmjyvo_lightcurve.csv
Loaded 18 data points after cleaning.
  Fit completed for ZTF18abmjyvo
Processing file 207/5003: ZTF18abmmdif_lightcurve.csv
Loaded 25 data points after cleaning.
  Fit completed for ZTF18abmmdif
Processing file 208/5003: ZTF18abmmkaz_lightcurve.csv
Loaded 21 data points after cleaning.
  Fit completed for ZTF18abmmkaz
Processing file 209/5003: ZTF18abmoung_lightcurve.csv
Loaded 20 data points after cleaning.
  Fit completed for ZTF18abmoung
Processing file 210/5003: ZTF18abmrhom_lightcurve.csv
Loaded 23 data points after cleaning.
  Fit completed for ZTF18abmrhom
Processing file 211/5003: ZTF18abmwnov_lightcurve.csv
Loaded 49 data points after cleaning.
  Fit completed for ZTF18abmwnov
Processing file 212/5003: ZTF18abmxahs_lightcurve.csv
Loaded 25 data points after cleaning.
  Fit completed for ZTF18abmxahs
Processing file 213/5003: ZTF18abmxdhb_lightcurve.csv
Loaded 40 data points after cleaning.


  Fit completed for ZTF18acfwmqj
Processing file 272/5003: ZTF18achxcsh_lightcurve.csv
Loaded 19 data points after cleaning.
  Fit completed for ZTF18achxcsh
Processing file 273/5003: ZTF18acjxyak_lightcurve.csv
Loaded 15 data points after cleaning.
  Fit completed for ZTF18acjxyak
Processing file 274/5003: ZTF18acmyxfx_lightcurve.csv
Loaded 12 data points after cleaning.
  Fit completed for ZTF18acmyxfx
Processing file 275/5003: ZTF18acnbenl_lightcurve.csv
Loaded 17 data points after cleaning.
  Fit completed for ZTF18acnbenl
Processing file 276/5003: ZTF18acnbeuu_lightcurve.csv
Loaded 35 data points after cleaning.
  Fit completed for ZTF18acnbeuu
Processing file 277/5003: ZTF18acnnxor_lightcurve.csv
Loaded 9 data points after cleaning.
  Fit completed for ZTF18acnnxor
Processing file 278/5003: ZTF18acpeemg_lightcurve.csv
Loaded 8 data points after cleaning.
  Fit completed for ZTF18acpeemg
Processing file 279/5003: ZTF18acptgzz_lightcurve.csv
Loaded 17 data points after cleaning.
  

...

  Fit completed for ZTF22abtbijr
Processing file 3251/5003: ZTF22abtbjbo_lightcurve.csv
Loaded 7 data points after cleaning.
  Fit completed for ZTF22abtbjbo
Processing file 3252/5003: ZTF22abtctfn_lightcurve.csv
Loaded 23 data points after cleaning.
  Fit completed for ZTF22abtctfn
Processing file 3253/5003: ZTF22abtctvo_lightcurve.csv
Loaded 22 data points after cleaning.
  Fit completed for ZTF22abtctvo
Processing file 3254/5003: ZTF22abtcyos_lightcurve.csv
Loaded 11 data points after cleaning.
  Fit completed for ZTF22abtcyos
Processing file 3255/5003: ZTF22abtdbnx_lightcurve.csv
Loaded 41 data points after cleaning.
  Fit completed for ZTF22abtdbnx
Processing file 3256/5003: ZTF22abtdoxf_lightcurve.csv
Loaded 18 data points after cleaning.
  Fit completed for ZTF22abtdoxf
Processing file 3257/5003: ZTF22abtdzyw_lightcurve.csv
Loaded 16 data points after cleaning.
  Fit completed for ZTF22abtdzyw
Processing file 3258/5003: ZTF22abteetl_lightcurve.csv
Loaded 23 data points after cle

  Fit completed for ZTF22abystdy
Processing file 3322/5003: ZTF22abzaijw_lightcurve.csv
Loaded 11 data points after cleaning.
  Fit completed for ZTF22abzaijw
Processing file 3323/5003: ZTF22abzfotk_lightcurve.csv
Loaded 20 data points after cleaning.
  Fit completed for ZTF22abzfotk
Processing file 3324/5003: ZTF22abzkion_lightcurve.csv
Loaded 10 data points after cleaning.
  Fit completed for ZTF22abzkion
Processing file 3325/5003: ZTF22abzuyxy_lightcurve.csv
Loaded 5 data points after cleaning.
  Fit completed for ZTF22abzuyxy
Processing file 3326/5003: ZTF22acablwr_lightcurve.csv
Loaded 22 data points after cleaning.
  Fit completed for ZTF22acablwr
Processing file 3327/5003: ZTF22acadszu_lightcurve.csv
Loaded 22 data points after cleaning.
  Fit completed for ZTF22acadszu
Processing file 3328/5003: ZTF22acalffo_lightcurve.csv
Loaded 6 data points after cleaning.
  Fit completed for ZTF22acalffo
Processing file 3329/5003: ZTF23aaabctl_lightcurve.csv
Loaded 18 data points after clea

  Fit completed for ZTF23aaeardk
Processing file 3392/5003: ZTF23aaeauii_lightcurve.csv
Loaded 18 data points after cleaning.
  Fit completed for ZTF23aaeauii
Processing file 3393/5003: ZTF23aaeavsh_lightcurve.csv
Loaded 26 data points after cleaning.
  Fit completed for ZTF23aaeavsh
Processing file 3394/5003: ZTF23aaebopi_lightcurve.csv
Loaded 12 data points after cleaning.
  Fit completed for ZTF23aaebopi
Processing file 3395/5003: ZTF23aaedrcn_lightcurve.csv
Loaded 28 data points after cleaning.
  Fit completed for ZTF23aaedrcn
Processing file 3396/5003: ZTF23aaedrzc_lightcurve.csv
Loaded 16 data points after cleaning.
  Fit completed for ZTF23aaedrzc
Processing file 3397/5003: ZTF23aaedsfn_lightcurve.csv
Loaded 20 data points after cleaning.
  Fit completed for ZTF23aaedsfn
Processing file 3398/5003: ZTF23aaeeawl_lightcurve.csv
Loaded 39 data points after cleaning.
  Fit completed for ZTF23aaeeawl
Processing file 3399/5003: ZTF23aaehsgs_lightcurve.csv
Loaded 35 data points after cl

  Fit completed for ZTF23aahfxpr
Processing file 3459/5003: ZTF23aahfzhm_lightcurve.csv
Loaded 34 data points after cleaning.
  Fit completed for ZTF23aahfzhm
Processing file 3460/5003: ZTF23aahkrlw_lightcurve.csv
Loaded 10 data points after cleaning.
  Fit completed for ZTF23aahkrlw
Processing file 3461/5003: ZTF23aahnabd_lightcurve.csv
Loaded 16 data points after cleaning.
  Fit completed for ZTF23aahnabd
Processing file 3462/5003: ZTF23aahvmpq_lightcurve.csv
Loaded 40 data points after cleaning.
  Fit completed for ZTF23aahvmpq
Processing file 3463/5003: ZTF23aahvnck_lightcurve.csv
Loaded 6 data points after cleaning.
  Fit completed for ZTF23aahvnck
Processing file 3464/5003: ZTF23aahxdhe_lightcurve.csv
Loaded 26 data points after cleaning.
  Fit completed for ZTF23aahxdhe
Processing file 3465/5003: ZTF23aahxdsi_lightcurve.csv
Loaded 29 data points after cleaning.
  Fit completed for ZTF23aahxdsi
Processing file 3466/5003: ZTF23aahxppo_lightcurve.csv
Loaded 14 data points after cle

  Fit completed for ZTF23aamfmqm
Processing file 3525/5003: ZTF23aamixrh_lightcurve.csv
Loaded 28 data points after cleaning.
  Fit completed for ZTF23aamixrh
Processing file 3526/5003: ZTF23aamlodm_lightcurve.csv
Loaded 14 data points after cleaning.
  Fit completed for ZTF23aamlodm
Processing file 3527/5003: ZTF23aamqonh_lightcurve.csv
Loaded 66 data points after cleaning.
  Fit completed for ZTF23aamqonh
Processing file 3528/5003: ZTF23aamqtel_lightcurve.csv
Loaded 26 data points after cleaning.
  Fit completed for ZTF23aamqtel
Processing file 3529/5003: ZTF23aamqzyz_lightcurve.csv
Loaded 61 data points after cleaning.
  Fit completed for ZTF23aamqzyz
Processing file 3530/5003: ZTF23aamtyjs_lightcurve.csv
Loaded 20 data points after cleaning.
  Fit completed for ZTF23aamtyjs
Processing file 3531/5003: ZTF23aamubjk_lightcurve.csv
Loaded 34 data points after cleaning.
  Fit completed for ZTF23aamubjk
Processing file 3532/5003: ZTF23aamvyyl_lightcurve.csv
Loaded 44 data points after cl